In [1]:
import pandas as pd
import time
start = time.time()
chunk = pd.read_csv('trans_and_named.csv',chunksize=1000000)
end = time.time()
print("Read csv with chunks: ",(end-start),"sec")
df = pd.concat(chunk)

Read csv with chunks:  0.06179642677307129 sec


/tmp/ipykernel_8066/2254671984.py:7: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.concat(chunk)


In [2]:
import ast
df['spacy_ner'] = df['spacy_ner'].apply(ast.literal_eval)
df['twit_ner'] = df['twit_ner'].apply(ast.literal_eval)
# Using regular expression to extract the lower and upper bounds
df[['lower_bound_spend', 'upper_bound_spend']] = df['spend'].str.extract('lower_bound: (\d+), upper_bound: (\d+)')
# Fill NaN values with 0
df['lower_bound_spend'].fillna(0, inplace=True)
df['upper_bound_spend'].fillna(0, inplace=True)
# Convert the extracted columns to integers
df['lower_bound_spend'] = df['lower_bound_spend'].astype(int)
df['upper_bound_spend'] = df['upper_bound_spend'].astype(int)

In [3]:
states_to_phase = {
    "ANDHRA PRADESH": 1,
    "ARUNACHAL PRADESH": 1,
    "ASSAM": [1, 2, 3],
    "BIHAR": [1, 2, 3, 4, 5, 6, 7],
    "CHATTISGARH": [1, 2, 3],
    "GOA": 3,
    "GUJARAT": 3,
    "HARYANA": 6,
    "HIMACHAL PRADESH": 7,
    "JAMMU & KASHMIR": [1, 2, 3, 4, 5],
    "JHARKHAND": [4, 5, 6, 7],
    "KARNATAKA": [2, 3],
    "KERALA": 3,
    "MADHYA PRADESH": [4, 5, 6, 7],
    "MAHARASHTRA": [3, 4],
    "MANIPUR": [1, 2],
    "MEGHALAYA": 1,
    "MIZORAM": 1,
    "NAGALAND": 1,
    "ODISHA": [1, 2, 3, 4],
    "PUNJAB": 7,
    "RAJASTHAN": [4, 5],
    "SIKKIM": 1,
    "TAMIL NADU": 2,
    "TELANGANA": 1,
    "TRIPURA": [1, 2],
    "UTTAR PRADESH": [1, 2, 3, 4, 5, 6, 7],
    "UTTARAKHAND": 1,
    "WEST BENGAL": [1, 2, 3, 4, 5, 6, 7],
    "ANDAMAN & NICOBAR ISLANDS": 1,
    "CHANDIGARH": 7,
    "DADRA & NAGAR HAVELI": 3,
    "DAMAN & DIU": 3,
    "LAKSHADWEEP": 1,
    "NATIONAL CAPITAL TERRITORY OF DELHI": 6,
    "PUDUCHERRY": 2
}
import datetime
phase_to_nomination_date = {1: datetime.datetime(2019, 3, 25, 0, 0),
 2: datetime.datetime(2019, 3, 26, 0, 0),
 3: datetime.datetime(2019, 4, 4, 0, 0),
 4: datetime.datetime(2019, 4, 9, 0, 0),
 5: datetime.datetime(2019, 4, 18, 0, 0),
 6: datetime.datetime(2019, 4, 23, 0, 0),
 7: datetime.datetime(2019, 4, 29, 0, 0)}
result_date=datetime.datetime(2019,5,23)
det=pd.read_csv("AllDetails.csv")

In [4]:
df['ad_delivery_start_time'] = pd.to_datetime(df['ad_delivery_start_time'], errors='coerce')
df['ad_delivery_stop_time'] = pd.to_datetime(df['ad_delivery_stop_time'], errors='coerce')

In [5]:
def filter_ads_by_date_range(df,d1,d2):
    valid_indices = set()
    for single_date in pd.date_range(start=d1, end=d2):
        rows_on_date = df[(df['ad_delivery_start_time'] <= single_date) & (df['ad_delivery_stop_time'] >= single_date)]
        valid_indices.update(rows_on_date.index)
    filtered_df = df.loc[valid_indices].reset_index(drop=True)
    return filtered_df

In [6]:
dfx=filter_ads_by_date_range(df,phase_to_nomination_date[1],result_date)

/tmp/ipykernel_8066/977046568.py:6: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  filtered_df = df.loc[valid_indices].reset_index(drop=True)


In [11]:
columns=["ad_archive_id","page_id","ad_delivery_start_time","ad_delivery_stop_time","translated page_name","translated ad_creative_bodies","ad_creative_link_descriptions","lower_bound_spend","upper_bound_spend"]
df_selected = dfx[columns]

In [13]:
df_selected.to_csv("to_render.csv")

In [7]:
det

,Name,Location,Party,SoDoWo,Name Enrolled As Voter In,Self Profession,Spouse Profession,Number of Criminal Cases,Assets,Liabilities,Pan Info,Brief Criminal Record,Pending Cases Data,Convicted Cases Data,Crimes Parsed Properly,Movable Assets,Immovable Assets,Contracts,Expenditure,Money Raised
0,V V KHALID,ANDAMAN AND NICOBAR ISLANDS (ANDAMAN & NICOB...,IND,V. V. Mohammed,Andaman and Nicobar Islands-26 (Andaman and Ni...,Private worker,Housewife,NaN,"Rs 15,12,000","{0: {0: 'Sr No', 1: 'i', 2: 'i', 3: 'i', 4: 'i...","{'self': {'PAN Given': 'Y', 'Financial Year': ...","{'Counts': [], 'Charges': []}",{},{},No Problems in Reading Affidavit Information,"{0: {0: 'Sr No', 1: 'i', 2: 'ii', 3: 'iii', 4:...","{0: {0: 'Sr No', 1: 'i', 2: 'ii', 3: 'iii', 4:...",{0: {0: 'Details of contracts entered by the c...,{0: {0: 'Expenditure Summary Whether accounts ...,{}
1,Vinod Prasad Notiyal,GARHWAL (UTTARAKHAND),IND,Bachhiram,"18 Dharampur (Uttarakhand) constituency, at Se...",Agriculturist,Agriculturist,NaN,"Rs 16,85,354","{0: {0: 'Sr No', 1: 'i', 2: 'i', 3: 'i', 4: 'i...","{'self': {'PAN Given': 'Y', 'Financial Year': ...","{'Counts': [], 'Charges': []}",{},{},No Problems in Reading Affidavit Information,"{0: {0: 'Sr No', 1: 'i', 2: 'ii', 3: 'iii', 4:...","{0: {0: 'Sr No', 1: 'i', 2: 'ii', 3: 'iii', 4:...",{0: {0: 'Details of contracts entered by the c...,{0: {0: 'Expenditure Summary Whether accounts ...,{}
2,WAHENGBAM POBITRA SINGH,INNER MANIPUR (MANIPUR),IND,W. Ibomcha Singh,"14 Yaiskul (Manipur) constituency, at Serial ...",LIC Agent,Home Maker,1.0,"Rs 1,30,022","{0: {0: 'Sr No', 1: 'i', 2: 'i', 3: 'i', 4: 'i...","{'self': {'PAN Given': 'Y', 'Financial Year': ...","{'Counts': ['1 ', ' 1 ', ' 1 ', ' 1 ', ' 1 '],...","{'Serial No.': [' 1'], 'FIR No.': [' 45(4) II ...","{'Serial No.': [' 1'], 'FIR No.': [' 45(4) II ...",No Problems in Reading Affidavit Information,"{0: {0: 'Sr No', 1: 'i', 2: 'ii', 3: 'iii', 4:...","{0: {0: 'Sr No', 1: 'i', 2: 'ii', 3: 'iii', 4:...",{0: {0: 'Details of contracts entered by the c...,{0: {0: 'Expenditure Summary Whether accounts ...,{}
3,VISHAL JOLLY,ANDAMAN AND NICOBAR ISLANDS (ANDAMAN & NICOB...,BJP,Late Brij Mohan Jolly,Andaman and Nicobar Islands (Andaman and Nicob...,Legal Practitioner,Govt. Employee,1.0,"Rs 8,51,40,573","{0: {0: 'Sr No', 1: 'i', 2: 'i', 3: 'i', 4: 'i...","{'self': {'PAN Given': 'Y', 'Financial Year': ...","{'Counts': [' 1 ', ' 1 '], 'Charges': [' charg...","{'Serial No.': [' 1'], 'FIR No.': [' 21/2011, ...","{'Serial No.': [' 1'], 'FIR No.': [' 21/2011, ...",No Problems in Reading Affidavit Information,"{0: {0: 'Sr No', 1: 'i', 2: 'ii', 3: 'iii', 4:...","{0: {0: 'Sr No', 1: 'i', 2: 'ii', 3: 'iii', 4:...",{0: {0: 'Details of contracts entered by the c...,{0: {0: 'Item of Expediture Expediture incurr...,"{0: {0: nan, 1: 'i', 2: 'ii', 3: 'iii', 4: 'ii..."
4,Thangminlien Kipgen,OUTER MANIPUR (MANIPUR),NPP,Late Thangpu Kipgen,"50- Kangpokpi (Manipur) constituency, at Seria...",Social Worker,Secretary Nagaland Secretariat Service Retired,NaN,"Rs 66,63,739","{0: {0: 'Sr No', 1: 'i', 2: 'i', 3: 'i', 4: 'i...","{'self': {'PAN Given': 'Y', 'Financial Year': ...","{'Counts': [], 'Charges': []}",{},{},No Problems in Reading Affidavit Information,"{0: {0: 'Sr No', 1: 'i', 2: 'ii', 3: 'iii', 4:...","{0: {0: 'Sr No', 1: 'i', 2: 'ii', 3: 'iii', 4:...",{0: {0: 'Details of contracts entered by the c...,{0: {0: 'Expenditure Summary Whether accounts ...,{}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7952,BHIKHLABHAI VANSABHAI KHULAT,DADRA AND NAGAR HAVELI (DADRA & NAGAR HAVELI),Bhartiya Tribal Party,Vansabhai Devaji Khulat,Dadra and Nagar Haveli (TU)-U03 (Dadra and Nag...,Junior Clerk - M.M.N. Dugad Gujarati High Scho...,NaN,1.0,Nil,{},"{'self': {'PAN Given': 'Y', 'Financial Year': ...","{'Counts': [], 'Charges': []}","{'Serial No.': [' 1'], 'FIR No.': [' Crime Reg...","{'Serial No.': [' 1'], 'FIR No.': [' Crime Reg...",No Problems in Reading Affidavit Information,{},{},

In [ ]:
import re
def extract_text_between(input_string, start_string, end_string):
    pattern = re.escape(start_string) + r'(.*?)' + re.escape(end_string)
    match = re.search(pattern, input_string, re.DOTALL)
    return match.group(1) if match else ''
def process_substring(input_string):
    no_commas = input_string.replace(',', '')
    words = no_commas.split()
    filtered_words = [word for word in words if 'xa0' not in word]
    clean_string = ' '.join(filtered_words)
    numbers = [int(num) for num in re.findall(r'\b\d+\b', clean_string)]
    if len(numbers)>0:
        return numbers[-1]
    else:
        return "NA"
parsed_spends=[]
for i in det["Expenditure"]:
    stry=extract_text_between(i,"Campaign through electronic/print media","Vehicles used and POL expenditure on such")
    fin=process_substring(stry)
    print(i,fin)
    parsed_spends.append(fin)

In [9]:
det["parsed_media"]=parsed_spends

In [10]:
det["parsed_media"].value_counts()

0         4331
NA        2081
1000        19
5000        15
20000       14
          ... 
5600         1
9010         1
400          1
30287        1
101245       1
Name: parsed_media, Length: 1284, dtype: int64

In [11]:
dfx

,Unnamed: 0,ad_archive_id,page_id,page_name,ad_creation_time,ad_delivery_start_time,ad_delivery_stop_time,byline,ad_creative_bodies,ad_creative_link_titles,...,publisher_platforms,estimated_audience_size,languages,translated page_name,translated ad_creative_bodies,percieved_translation,spacy_ner,twit_ner,lower_bound_spend,upper_bound_spend
0,542590,2288655964752962,1709897795958801,குமாரபாளையம் சட்டமன்ற தொகுதி பா.ஜ.க,2016-03-22,2018-04-28,2020-02-06,NaN,Vote for BJP to bring Tamilnadu to development...,குமாரபாள?,...,facebook,NaN,NaN,Kumarapalayam Assembly Constituency பா.ஜ.க,Vote for BJP to bring Tamilnadu to development...,English,[],[],0,99
1,542598,513244419493919,1534829196769126,Sachchidanand Rai,2016-09-30,2018-01-10,2019-04-20,NaN,Member - Bihar Legislative Council \nChairman ...,NaN,...,facebook,NaN,NaN,Sachchidanand Rai,Member - Bihar Legislative Council \nChairman ...,English,[],[],15000,19999
2,542604,2079476525498882,253441185022635,PMC BJP Campaign and Updates,2016-11-08,2019-03-27,2019-03-28,NaN,NaN,NaN,...,facebook,NaN,NaN,PMC BJP Campaign and Updates,NaN,English,[],[],0,99
3,542605,383306825584827,329683907650,lawmanpg3,2016-11-11,2019-03-05,2019-11-29,NaN,"An initiative towards making #India ""The Golde...",NaN,...,facebook,NaN,NaN,lawmanpg3,"An initiative towards making #India ""The Golde...",English,[],[ #PMNarendraModi],10000,14999
4,542606,2308261329428010,329683907650,lawmanpg3,2016-11-11,2019-03-05,2019-11-29,NaN,"An initiative towards making #India ""The Golde...",NaN,...,facebook,NaN,NaN,lawmanpg3,"An initiative towards making #India ""The Golde...",English,[],[ #PMNarendraModi],25000,29999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83226,708416,737506496651305,1068762883209737,Molitics,2019-05-23,2019-05-23,2019-05-25,Molitics,आम आदमी पार्टी के नेता आज दिल्ली के वोटर्स से ...,दिल्ली में AAP ने खोया जनाधार - Kejriwal की ये...,...,facebook,NaN,hi,Molitics,This is what the leaders of the Aam Aadmi Part...,hin_Deva,[],[],400,499
83227,708417,1090146744517828,1068762883209737,Molitics,2019-05-23,2019-05-23,2019-05-25,Molitics,दिल्ली से आम आदमी पार्टी का सूपड़ा साफ।\n#Resu...,केजरीवाल कैसे हैं आम आदमी पार्टी के पतन के ज़ि...,...,"facebook,instagram",NaN,hi,Molitics,Aam Aadmi Party sweeps from Delhi. #ResultWith...,hin_Deva,[],[],200,299
83228,708418,2052195195082708,137957159747076,Tarun Rathi,2019-05-23,2019-05-23,2019-05-24,Tarun Kumar Ratan Singh Rathi,NaN,NaN,...,facebook,NaN,hi,Tarun Rathi,NaN,English,[],[],0,99
83229,708419,2438244596399098,973189589455249,Thakur Jaivir Singh,2019-05-23,2019-05-23,2019-05-24,Thakur Jaivir Singh,"यह जीत पूरे भारत की जीत है।\nदेश के युवा, गरीब...",NaN,...,"facebook,instagram",NaN,hi,Thakur Jaivir Singh,This victory is the victory of the whole of In...,hin_Deva,"[Modi ji, Narendra Modi]","[ Prime Minister, Shri Narendra Modi ji, Mod...",100,199


In [12]:
det

,Name,Location,Party,SoDoWo,Name Enrolled As Voter In,Self Profession,Spouse Profession,Number of Criminal Cases,Assets,Liabilities,...,Brief Criminal Record,Pending Cases Data,Convicted Cases Data,Crimes Parsed Properly,Movable Assets,Immovable Assets,Contracts,Expenditure,Money Raised,parsed_media
0,V V KHALID,ANDAMAN AND NICOBAR ISLANDS (ANDAMAN & NICOB...,IND,V. V. Mohammed,Andaman and Nicobar Islands-26 (Andaman and Ni...,Private worker,Housewife,NaN,"Rs 15,12,000","{0: {0: 'Sr No', 1: 'i', 2: 'i', 3: 'i', 4: 'i...",...,"{'Counts': [], 'Charges': []}",{},{},No Problems in Reading Affidavit Information,"{0: {0: 'Sr No', 1: 'i', 2: 'ii', 3: 'iii', 4:...","{0: {0: 'Sr No', 1: 'i', 2: 'ii', 3: 'iii', 4:...",{0: {0: 'Details of contracts entered by the c...,{0: {0: 'Expenditure Summary Whether accounts ...,{},NA
1,Vinod Prasad Notiyal,GARHWAL (UTTARAKHAND),IND,Bachhiram,"18 Dharampur (Uttarakhand) constituency, at Se...",Agriculturist,Agriculturist,NaN,"Rs 16,85,354","{0: {0: 'Sr No', 1: 'i', 2: 'i', 3: 'i', 4: 'i...",...,"{'Counts': [], 'Charges': []}",{},{},No Problems in Reading Affidavit Information,"{0: {0: 'Sr No', 1: 'i', 2: 'ii', 3: 'iii', 4:...","{0: {0: 'Sr No', 1: 'i', 2: 'ii', 3: 'iii', 4:...",{0: {0: 'Details of contracts entered by the c...,{0: {0: 'Expenditure Summary Whether accounts ...,{},NA
2,WAHENGBAM POBITRA SINGH,INNER MANIPUR (MANIPUR),IND,W. Ibomcha Singh,"14 Yaiskul (Manipur) constituency, at Serial ...",LIC Agent,Home Maker,1.0,"Rs 1,30,022","{0: {0: 'Sr No', 1: 'i', 2: 'i', 3: 'i', 4: 'i...",...,"{'Counts': ['1 ', ' 1 ', ' 1 ', ' 1 ', ' 1 '],...","{'Serial No.': [' 1'], 'FIR No.': [' 45(4) II ...","{'Serial No.': [' 1'], 'FIR No.': [' 45(4) II ...",No Problems in Reading Affidavit Information,"{0: {0: 'Sr No', 1: 'i', 2: 'ii', 3: 'iii', 4:...","{0: {0: 'Sr No', 1: 'i', 2: 'ii', 3: 'iii', 4:...",{0: {0: 'Details of contracts entered by the c...,{0: {0: 'Expenditure Summary Whether accounts ...,{},NA
3,VISHAL JOLLY,ANDAMAN AND NICOBAR ISLANDS (ANDAMAN & NICOB...,BJP,Late Brij Mohan Jolly,Andaman and Nicobar Islands (Andaman and Nicob...,Legal Practitioner,Govt. Employee,1.0,"Rs 8,51,40,573","{0: {0: 'Sr No', 1: 'i', 2: 'i', 3: 'i', 4: 'i...",...,"{'Counts': [' 1 ', ' 1 '], 'Charges': [' charg...","{'Serial No.': [' 1'], 'FIR No.': [' 21/2011, ...","{'Serial No.': [' 1'], 'FIR No.': [' 21/2011, ...",No Problems in Reading Affidavit Information,"{0: {0: 'Sr No', 1: 'i', 2: 'ii', 3: 'iii', 4:...","{0: {0: 'Sr No', 1: 'i', 2: 'ii', 3: 'iii', 4:...",{0: {0: 'Details of contracts entered by the c...,{0: {0: 'Item of Expediture Expediture incurr...,"{0: {0: nan, 1: 'i', 2: 'ii', 3: 'iii', 4: 'ii...",173908
4,Thangminlien Kipgen,OUTER MANIPUR (MANIPUR),NPP,Late Thangpu Kipgen,"50- Kangpokpi (Manipur) constituency, at Seria...",Social Worker,Secretary Nagaland Secretariat Service Retired,NaN,"Rs 66,63,739","{0: {0: 'Sr No', 1: 'i', 2: 'i', 3: 'i', 4: 'i...",...,"{'Counts': [], 'Charges': []}",{},{},No Problems in Reading Affidavit Information,"{0: {0: 'Sr No', 1: 'i', 2: 'ii', 3: 'iii', 4:...","{0: {0: 'Sr No', 1: 'i', 2: 'ii', 3: 'iii', 4:...",{0: {0: 'Details of contracts entered by the c...,{0: {0: 'Expenditure Summary Whether accounts ...,{},NA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7952,BHIKHLABHAI VANSABHAI KHULAT,DADRA AND NAGAR HAVELI (DADRA & NAGAR HAVELI),Bhartiya Tribal Party,Vansabhai Devaji Khulat,Dadra and Nagar Haveli (TU)-U03 (Dadra and Nag...,Junior Clerk - M.M.N. Dugad Gujarati High Scho...,NaN,1.0,Nil,{},...,"{'Counts': [], 'Charges': []}","{'Serial No.': [' 1'], 'FIR No.': [' Crime Reg...","{'Serial No.': [' 1'], 'FIR No.': [' Crime Reg...",No Problems in Reading Affidavit Information,{},{},{0: {0: 'Details of contracts entered by the c...,{0: {0: 'Item of Expediture Expediture incurr...,"{0: {0: nan, 1: 'i', 2: 'ii', 3: 'iii', 4: 'ii...",0
7953,HASIM AKHTAR,MALDAHA DAKSHIN (WEST BENGAL),IND,Late Muktar Mia,"51 English Bazar (West Bengal) cons

In [13]:
import pandas as pd
def create_combined_text_column(df, text_columns, new_column_name):
    for col in text_columns:
        if col not in df.columns:
            raise ValueError(f"Column '{col}' not found in DataFrame")
    df[new_column_name] = df[text_columns].astype(str).agg(' '.join, axis=1)
    return df
dfx=create_combined_text_column(dfx,["translated page_name","translated ad_creative_bodies","ad_creative_link_descriptions"],"alltext")
def filter_by_string(df, column, string):
    df[column] = df[column].fillna('')
    lowercased_col = df[column].str.lower()
    lowercased_string = string.lower()
    filtered_df = df[lowercased_col.str.contains(lowercased_string, regex=False)]
    return filtered_df


In [14]:
import pandas as pd
from tqdm import tqdm 
def preprocess_name(name):
    return ' '.join([word for word in name.split() if len(word) > 1]).lower()
det['processed_name'] = det['Name'].apply(preprocess_name)
article_to_spending = dict(zip(dfx['alltext'].str.lower(), dfx['upper_bound_spend']))
results = {}
for index, row in tqdm(det.iterrows(), total=det.shape[0]):
    processed_name = row['processed_name']
    declared_spending = row['parsed_media']
    calculated_spending = 0
    filtered_articles = filter_by_string(dfx, 'alltext', processed_name)
    results[processed_name] = {
        'actual_name': row['Name'],
        'declared_spending': declared_spending,
        'calculated_spending': filtered_articles["upper_bound_spend"].sum()
    }
results_df = pd.DataFrame.from_dict(results, orient='index').reset_index(drop=True)

  0%|          | 1/7957 [00:00<23:52,  5.55it/s]

100%|██████████| 7957/7957 [24:01<00:00,  5.52it/s]


In [15]:
import pandas as pd
data_to_convert = []
for processed_name, spending_data in results.items():
    spending_data['processed_name'] = processed_name
    data_to_convert.append(spending_data)
dfn = pd.DataFrame(data_to_convert)
dfn

,actual_name,declared_spending,calculated_spending,processed_name
0,V V KHALID,NA,298,khalid
1,Vinod Prasad Notiyal,NA,0,vinod prasad notiyal
2,WAHENGBAM POBITRA SINGH,NA,0,wahengbam pobitra singh
3,VISHAL JOLLY,173908,0,vishal jolly
4,Thangminlien Kipgen,NA,299,thangminlien kipgen
...,...,...,...,...
7547,BHIKHLABHAI VANSABHAI KHULAT,0,0,bhikhlabhai vansabhai khulat
7548,HASIM AKHTAR,0,0,hasim akhtar
7549,DAYAT MANOJ RANCHOD,0,0,dayat manoj ranchod
7550,AJIT PRATAP KUSHAWAHA,7000,0,ajit pratap kushawaha


In [ ]:
ctr=0
for name, spending in results.items():
    if spending['declared_spending'] != 'NA' and spending['calculated_spending'] > spending['declared_spending']:
        print(f"{name}: Declared - {spending['declared_spending']}, Calculated - {spending['calculated_spending']}")
        ctr+=1

In [17]:
ctr

610

In [18]:
dfn

,actual_name,declared_spending,calculated_spending,processed_name
0,V V KHALID,NA,298,khalid
1,Vinod Prasad Notiyal,NA,0,vinod prasad notiyal
2,WAHENGBAM POBITRA SINGH,NA,0,wahengbam pobitra singh
3,VISHAL JOLLY,173908,0,vishal jolly
4,Thangminlien Kipgen,NA,299,thangminlien kipgen
...,...,...,...,...
7547,BHIKHLABHAI VANSABHAI KHULAT,0,0,bhikhlabhai vansabhai khulat
7548,HASIM AKHTAR,0,0,hasim akhtar
7549,DAYAT MANOJ RANCHOD,0,0,dayat manoj ranchod
7550,AJIT PRATAP KUSHAWAHA,7000,0,ajit pratap kushawaha


In [19]:
dfn['declared_spending_numeric'] = pd.to_numeric(dfn['declared_spending'], errors='coerce')
dfn

,actual_name,declared_spending,calculated_spending,processed_name,declared_spending_numeric
0,V V KHALID,NA,298,khalid,NaN
1,Vinod Prasad Notiyal,NA,0,vinod prasad notiyal,NaN
2,WAHENGBAM POBITRA SINGH,NA,0,wahengbam pobitra singh,NaN
3,VISHAL JOLLY,173908,0,vishal jolly,173908.0
4,Thangminlien Kipgen,NA,299,thangminlien kipgen,NaN
...,...,...,...,...,...
7547,BHIKHLABHAI VANSABHAI KHULAT,0,0,bhikhlabhai vansabhai khulat,0.0
7548,HASIM AKHTAR,0,0,hasim akhtar,0.0
7549,DAYAT MANOJ RANCHOD,0,0,dayat manoj ranchod,0.0
7550,AJIT PRATAP KUSHAWAHA,7000,0,ajit pratap kushawaha,7000.0


In [20]:
filtered_df = dfn[(dfn['declared_spending'] != "NA") & (dfn['declared_spending_numeric'] < dfn['calculated_spending'])]

In [21]:
filtered_df['spending_difference'] = filtered_df['declared_spending_numeric'] - filtered_df['calculated_spending']
sorted_df = filtered_df.sort_values(by='spending_difference', ascending=True)


/tmp/ipykernel_22912/3961378542.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['spending_difference'] = filtered_df['declared_spending_numeric'] - filtered_df['calculated_spending']


In [22]:
sorted_df

,actual_name,declared_spending,calculated_spending,processed_name,declared_spending_numeric,spending_difference
1296,BHARAT,0,41546176,bharat,0.0,-41546176.0
3602,Mini S,6500,16486811,mini,6500.0,-16480311.0
1666,CHO V,0,15455471,cho,0.0,-15455471.0
4722,NARENDRA,0,6757486,narendra,0.0,-6757486.0
2172,NAVEEN,0,6671844,naveen,0.0,-6671844.0
...,...,...,...,...,...,...
6789,RAUSHAN KUMAR,0,99,raushan kumar,0.0,-99.0
2613,HARJINDER SINGH KAKA SRAN,0,99,harjinder singh kaka sran,0.0,-99.0
6906,PANKAJ KUMAR,0,99,pankaj kumar,0.0,-99.0
7075,DHARMENDRA SINGH,0,99,dharmendra singh,0.0,-99.0


In [103]:
#first 25
sorted_df[125:150]

,actual_name,declared_spending,calculated_spending,processed_name,declared_spending_numeric,spending_difference
128,Smriti Irani,57000,118068,smriti irani,57000.0,-61068.0
864,KULDEEP,0,59843,kuldeep,0.0,-59843.0
912,KESHAV,0,58996,keshav,0.0,-58996.0
2019,SUKHBIR SINGH BADAL,18478,75458,sukhbir singh badal,18478.0,-56980.0
417,RAJVEER,0,56567,rajveer,0.0,-56567.0
580,SARAVANAN M,0,55864,saravanan,0.0,-55864.0
5368,SUKHBIR SINGH,20000,75458,sukhbir singh,20000.0,-55458.0
1968,MAUSAM NOOR,0,54495,mausam noor,0.0,-54495.0
5397,RANJEET SINGH,0,53784,ranjeet singh,0.0,-53784.0
5716,Subhash Chand,0,52755,subhash chand,0.0,-52755.0


In [116]:
tempx=filter_by_string(dfx,"alltext","kalyan chaubey")
print(len(tempx))
print(tempx["upper_bound_spend"].sum())

32
50368


In [117]:
tempx["alltext"].sample(30,replace=True).tolist()

['Kalyan Chaubey BJP candidate from Krishnanagar Lok Sabha constituency, Chowkidar Kalyan Choubey Kalyan Chaubey has played Professional Football in the Indian Team for 7 years and top I-league clubs for 15 years.\nAparinoto is First novel by Kalyan Chaubey as an author.\nKalyan Chaubey is a BJP Loksabha candidate from Krishnanagar, WB.',
 'Kalyan Chaubey BJP candidate from Krishnanagar Lok Sabha constituency, Chowkidar Kalyan Choubey Kalyan Chaubey has played Professional Football in the Indian Team for 7 years and top I-league clubs for 15 years.\nAparinoto is First novel by Kalyan Chaubey as an author.\nKalyan Chaubey is a BJP Loksabha candidate from Krishnanagar, WB.',
 'Arjun Singh The bloody politics that Mamata didi and the TMC goons are playing in Bengal, the public is watching and will punish them by expelling them from Bengal. Never in the history of India has there been such bloody politics and the public has never supported such politics. The time has come for Mamata didi t